In [4]:

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from pythreejs import *
import cv2

calibration_file_path = "C:/Users/leeju/OneDrive/바탕 화면/kitti_object_vis-master/kitti_object_vis-master/data/object/training/calib/000000.txt"
image_file_path = "C:/Users/leeju/OneDrive/바탕 화면/kitti_object_vis-master/kitti_object_vis-master/data/object/training/image_2/000000.png"
lidar_data_path = "C:/Users/leeju/OneDrive/바탕 화면/kitti_object_vis-master/kitti_object_vis-master/data/object/training/velodyne/000000.bin"

stream = open(image_file_path, "rb")
bytes = bytearray(stream.read())
numpyarray = np.asarray(bytes, dtype=np.uint8)
image_color = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
image_color = image_color[:, :, (2, 1, 0)]  # BGR -> RGB
print(image_color.shape)

# Read calibration data
with open(calibration_file_path, 'r') as fid:
    lines = fid.readlines()
    #앞뒤 공백을 지우고, 띄어쓰기 기준으로 라인을 나눠 배열로 저장
    for i in range(7):
        lines[i] = lines[i].strip().split(' ')
        #첫 번째 P0: 이런 것들 다 지워준다
        del lines[i][0]
        # 각 배열의 원소들을 한 칸씩 띄워 한 문자열로 저장
        lines[i] = ' '.join(lines[i])

    # 문자열로부터 숫자 데이터를 뽑아 배열로 저장. 분류 기준은 띄어쓰기다.(문자 -> 숫자)
    P0 = np.fromstring(lines[0], dtype=float, sep=' ')
    P1 = np.fromstring(lines[1], dtype=float, sep=' ')
    P2 = np.fromstring(lines[2], dtype=float, sep=' ')
    P3 = np.fromstring(lines[3], dtype=float, sep=' ')
    R0_rect = np.fromstring(lines[4], dtype=float, sep=' ')
    Tr_velo_to_cam = np.fromstring(lines[5], dtype=float, sep=' ')
    Tr_imu_to_velo = np.fromstring(lines[6], dtype=float, sep=' ')
    Tr_cam_to_road = np.fromstring(lines[7], dtype=float, sep=' ')

#Read LiDAR data
data = np.fromfile(lidar_data_path, dtype=np.float32).reshape(-1, 4)
# Perform LiDAR-to-Image mapping
R0=R0_rect.reshape(3,3)
Tr=Tr_velo_to_cam.reshape(3,4)
p2=P2.reshape(3,4)
XYZ=data[:,:3].T

XYZ1 = np.vstack((XYZ, np.ones(data.shape[0])))
endgame=np.dot(R0, np.dot(Tr, XYZ1))
real_endgame=np.vstack((endgame, np.ones(endgame.T.shape[0])))
xy1=np.dot(p2,real_endgame)

p2=P2.reshape(3,4)
print(p2)

#openCV의 decomposeProjectionMatrix 함수를 통해 K,R,T를 분리해준다.
k2,r2,t2,_,_,_,_= cv2.decomposeProjectionMatrix(p2)
print(k2)
print(r2)

#t2의 element가 4개인 이유는, decompose를 할 때 4X4 size를 기준으로 T를 잘라냈기 때문이다. 그렇기에 마지막 0.998은 없어도 된다.
print(t2)
t2=t2[:3,0]
print(t2)

weight = xy1[2, :]
x=xy1[0,:]/weight
y=xy1[1,:]/weight
k = np.where(weight > 0)[0]
valid_xy1=xy1.T[k]

#world coordinate system에서의 point cloud 정보들
valid_xy1=np.dot(np.linalg.inv(k2),valid_xy1.T)
valid_xy1=valid_xy1.T-t2
valid_xy1=np.dot(np.linalg.inv(r2),valid_xy1.T)
depth_camera02=valid_xy1.T

'''
x=x[k].reshape(-1,1)
y=y[k].reshape(-1,1)
w=weight[k].reshape(-1,1)
xy=np.hstack((x,y))
xy=np.hstack((xy,w))
sorted_indices = np.argsort(xy[:, 0])[::1]
sorted_xy = xy[sorted_indices]
img_x_min = min(np.where(sorted_xy[:,0]>= 0)[0])
img_x_max = max(np.where(sorted_xy[:,0]<= img.width)[0])
sorted_xy=sorted_xy[img_x_min:img_x_max,:]
sorted_xy = sorted_xy[:, [1, 0,2]]
print('sorted_xy',sorted_xy)
sorted_indices = np.argsort(sorted_xy[:, 0])[::1]
sorted_xy = sorted_xy[sorted_indices]
img_y_min = min(np.where(sorted_xy[:,0]>= 0)[0])
img_y_max = max(np.where(sorted_xy[:,0]<= img.height)[0])
sorted_xy=sorted_xy[img_y_min:img_y_max,:]
new_xy = sorted_xy[:, [1, 0,2]]
print(new_xy.shape)
'''



'''
사진 사이즈에 맞춰서 point cloud 잘라내는 과정
위에 주석처리 한 것처럼 형이 했던대로 올림차순으로 정렬한다음에 정리하면 index가 뒤죽박죽돼서 복잡해지는 관계로!
이렇게 노가다식으로 잘라냄 (그래서 형이 한 방식보다 2개 더 많은 20285개...가 있음. 즉 이 방식은 조오금 부정확하긴하다는 뜻)
'''
x=x[k].reshape(-1,1)
y=y[k].reshape(-1,1)
w=weight[k].reshape(-1,1)
xy=np.hstack((x,y))

xy_index11=np.where(xy[:,0]<1224)
xy_index12=np.where(xy[:,0]>=0)
xy_index21=np.where(xy[:,1]<370)
xy_index22=np.where(xy[:,1]>=0)
xy_index1=np.intersect1d(xy_index11,xy_index12)
xy_index2=np.intersect1d(xy_index21,xy_index22)

#xy_index: 카메라가 바라보는 범위 안에 들어오는 point cloud들에 대한 index
xy_index=np.intersect1d(xy_index1,xy_index2)
reshaped_xy=xy[xy_index]
print("reshaped_xy_shape",reshaped_xy.shape)

RGB_to_world=np.zeros((1224*370,2)).reshape(-1,2)

#pixel coordinate에서 사진의 (x,y)좌표를 matrix 형태로 저장
for i in range(370):
    for j in range(1224):
        RGB_to_world[i*1224+j,0]=i
        RGB_to_world[i*1224+j,1]=j
image_color=image_color.reshape(-1,3)
print(image_color.shape)

#픽셀의 (x,y) 좌표와 (r,g,b)정보를 hstack해준다.
rgb=np.hstack((RGB_to_world,image_color))
print(rgb)

#픽셀의 각 좌표 정보는 전부 정수이기 때문에, 한 픽셀 내부에 속한 점들은 모두 같은 색을 띌 수 있도록 해준다.
int_xy=reshaped_xy.astype(int)
rgbbb=np.zeros((len(int_xy.T[0]),3))
for i in range(len(int_xy.T[0])):
    indexx=int_xy[i,1]*1224+int_xy[i,0]
    rgbbb[i]=rgb[indexx,2:6]
print(rgbbb)

print(reshaped_xy)
print(reshaped_xy.shape)

image_point_tmp=np.vstack((reshaped_xy.T,np.ones(reshaped_xy.T[0].shape)))

image_point_tmp=np.dot(np.linalg.inv(k2),image_point_tmp)
image_point_tmp=image_point_tmp.T-t2
image_point_tmp=np.dot(np.linalg.inv(r2),image_point_tmp.T)

#image_point를 display하면? 사진의 상이 맻히는 거리에 사진의 형태로 point cloud가 찍힌다.
image_point=np.hstack((image_point_tmp.T,rgbbb))
print(image_point)

#image_point2를 display하면? 우리가 원하는 형태의 point cloud를 얻을 수 있다.
image_point2=np.hstack((depth_camera02[xy_index],rgbbb))
print(image_point2)

Fov= 2*math.atan(p2[1,1]/p2[0,0])*(180/pi)
print(Fov)

normalized_RGB_color = image_point2 / 255.0
geometry = BufferGeometry(
     attributes={
        'position': BufferAttribute(image_point2[:,:3], normalized=False),
        'color': BufferAttribute(normalized_RGB_color[:,3:6], normalized=True)
     }
 )

material = PointsMaterial(size=0.1, vertexColors='VertexColors')
point=Points(geometry=geometry, material=material)

camera = PerspectiveCamera(up=[0,0,1], position=[0,-10,-10], near=0.01, aspect=400/300)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()
scene=Scene(children=[point,camera,key_light,ambient_light], background=None)
scene.add(AxesHelper(size=3))

lidar_position=np.dot(R0,np.dot(Tr,[0,0,0,1]))
print(lidar_position)

#Lidar sensor를 표현해주기 위한 PointLightHelper
point_light = PointLight(color="#ffffff", intensity=1, distance=100)
lidar_position=(lidar_position[0],lidar_position[1],lidar_position[2])
point_light_helper = PointLightHelper(point_light, position=lidar_position, distance=0.1, sphereSize=0.1)
scene.add(point_light)
scene.add(point_light_helper)

cam_position=[0,0,0]
cam_position=np.dot(np.linalg.inv(k2),cam_position)
cam_position=cam_position-t2
cam_position=np.dot(np.linalg.inv(r2),cam_position)
print(cam_position)

near_condition=np.where(image_point2[:,0]>=0)
near=np.min(image_point2[near_condition,0])
far=np.max(image_point2[:,0])

#near값을 확인해보면 매우 작은 걸 알 수 있다! 이걸로는 near를 표현하기 조금...
print(near)
print(far)

#image_point의 depth값의 의미하는 바: 카메라의 상이 맺히는 거리!(맞나?) 이 값을 near로 잡아주면 어떨까?
#뭔가 애매하긴 하지만, near의 기준으로 잡을만한 게 지금으로썬 없는 거 같다.
print(np.min(image_point[:,2]))
print(np.max(image_point[:,2]))
near=np.max(image_point[:,2])

#생각해보니 pythreejs의 axihelper하고 카메라 axis가 다르니 position을 다시 고려해야할듯 
cam=PerspectiveCamera(up=[0,0,-1], position=(cam_position[0],cam_position[1],cam_position[2]), aspect=370/1224, fov=Fov, near=near, far=far)
camera_helper = CameraHelper(cam)
camera_helper.rotateX(math.pi)
camera_helper.rotateZ(math.pi/2)
scene.add(camera_helper)

controller=OrbitControls(controlling=camera)
renderer=Renderer(camera=camera, scene=scene, controls=[controller],width=800,height=600)
display(renderer)

(370, 1224, 3)
[[ 7.070493e+02  0.000000e+00  6.040814e+02  4.575831e+01]
 [ 0.000000e+00  7.070493e+02  1.805066e+02 -3.454157e-01]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  4.981016e-03]]
[[707.0493   0.     604.0814]
 [  0.     707.0493 180.5066]
 [  0.       0.       1.    ]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[-0.06035061]
 [ 0.00175693]
 [-0.00497187]
 [ 0.99816331]]
[-0.06035061  0.00175693 -0.00497187]
reshaped_xy_shape (20285, 2)
(452880, 3)
[[0.000e+00 0.000e+00 1.700e+01 2.200e+01 1.800e+01]
 [0.000e+00 1.000e+00 1.600e+01 2.400e+01 2.800e+01]
 [0.000e+00 2.000e+00 2.500e+01 2.900e+01 3.300e+01]
 ...
 [3.690e+02 1.221e+03 2.200e+01 3.000e+01 3.700e+01]
 [3.690e+02 1.222e+03 2.200e+01 2.900e+01 3.800e+01]
 [3.690e+02 1.223e+03 2.000e+01 3.000e+01 3.500e+01]]
[[ 11.  17.  23.]
 [ 21.  24.  21.]
 [ 17.  32.  48.]
 ...
 [187. 187. 168.]
 [192. 185. 176.]
 [185. 187. 180.]]
[[602.0853193  141.7459889 ]
 [599.8489135  141.81345282]
 [596.12144246 149.02292823]
 ...
 [614.663

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, near=0.01, position=(0.0, -10.0, -10.0), projecti…